# Setup
Load data from DB into pandas dataframe

In [61]:
# Variables
dataset = "ar" #Choose ar (arabic) or en (english)
export_directory = "" # Set a path for you exports
database_file = f"data_{dataset}.db" # You can replace this with a path to your own db file


In [64]:

import plotly.express as px
import pandas as pd
import sqlite3
import nltk
import geostring as geo
from collections import Counter
import os

# Make sure export directory exists
full_export_dir = os.path.join(os.path.join(os.path.join(export_directory, "export"), dataset))
os.makedirs(full_export_dir, exist_ok=True)

lang_map = {"en": "english", "ar": "arabic"}

nltk.download("stopwords")
conn = sqlite3.connect(database_file)

tweets = pd.read_sql("SELECT * FROM tweets", conn)
tweets["timestamp"] = pd.to_datetime(tweets["timestamp"])
tweets["hashtags"] = tweets["hashtags"].str.split(",")

users = pd.read_sql("SELECT * FROM users", conn)

conn.close()
def get_country(location: str):
    g = geo.resolve(location or "")
    if g:
        return g["resolved_country"].split("?")[0]
    return None

users["country"] = users["location"].apply(lambda x: get_country(x) ) # This is very slow. Should find a better way

tweets_with_user = tweets.join(users.set_index('id'), on="author", lsuffix="_tweet", rsuffix="_user").set_index("id")


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/scullion/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tweet Timeline
Timeline of tweets (posted by verified and unverified accounts): the number of tweets per week. 

In [66]:
tweets_with_user = tweets.join(users.set_index('id'), on="author", lsuffix="_tweet", rsuffix="_user").set_index("id")
tweet_timeline_df = tweets_with_user.groupby([pd.Grouper(key="timestamp", freq="W-MON"), "verified"])["timestamp"].size().reset_index(name="counts")
tweet_timeline_df["verified"] = tweet_timeline_df["verified"].astype(bool)
fig = px.bar(tweet_timeline_df, x="timestamp", y="counts", 
       color="verified", 
       title="Timeline of tweets (posted by verified and unverified accounts)",
       labels={
           "counts": "Number Of Tweets",
           "timestamp": "Date"
           
       },
       hover_name="timestamp", 
       text="counts"
       )
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.write_image(os.path.join(full_export_dir, "tweet_timeline.png"))
fig.show()

# Top 20 Users
Top 20 users (based on the number of tweets they posted) + the number of tweets per user

In [4]:
top_users = tweets_with_user.groupby(["username"])["timestamp"].size().reset_index(name="counts").sort_values(by="counts", ascending=False)[0:20]
fig = px.bar(top_users,
             x="counts", 
             y="username", 
             orientation="h", 
             text="counts",
             labels={
                 "counts": "Number Of Tweets",
                 "username": "Twitter Account"}
             )
fig.update_traces( textposition='outside')
fig.update_yaxes(categoryorder='total ascending')
fig.write_image(os.path.join(full_export_dir, "top_users.png"))

fig.show()

# Top 10 Hashtags
The top 10 most hashtags and their time evolution (i.e., their frequency per week)

In [10]:
# Find the top 10 most common hashtags
top_hashtags = [i[0] for i in Counter(tweets[~tweets["hashtags"].isnull()]["hashtags"].explode()).most_common(10)]

# remove any hashtags that are not in the top dataset
tweets["top_hashtags"] = tweets["hashtags"].apply(lambda x: list(set(x if x else []) & set(top_hashtags)))

# Group data by week
hastag_timeline = tweets.groupby([pd.Grouper(key="timestamp", freq="W-MON")],)["top_hashtags"].sum().apply(lambda x: x if isinstance(x, list) else []).apply(Counter)

# reduce dataset to timeline with hashtag mentions
out = pd.DataFrame([(t, "#"+k, v) for t, d in zip(hastag_timeline.index.values, hastag_timeline) for k, v in d.items()])
out.columns = ['Date', 'Hashtags', 'Count']


fig = px.line(out, x="Date", y="Count", color="Hashtags")
fig.write_image(os.path.join(full_export_dir, "top_hashtags.png"))
fig.show()


## Top Words
The top 10 most frequent words (unigrams) and their time evolution (i.e., their frequency per week)

In [8]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize

# Use English stemmer.

stemmer = SnowballStemmer(lang_map[dataset])
cachedStopWords = stopwords.words(lang_map[dataset])

re_pattern = r"(@\[A-Za-z\u0621-\u064A\u0660-\u06690-9]+)|([^A-Za-z\u0621-\u064A\u0660-\u06690-9 \t])|(\w+:\/\/\S+)|^rt|http.+?"

# stem all non words from content and stem words
tweets["stemmed"] = tweets["tweet_text"].apply(lambda x: re.sub(re_pattern, "", x)).str.split(" |_|-").apply(lambda x: [stemmer.stem(y.strip().lower()) for y in x])

# remove stop words
tweets["clean_words"] = tweets["stemmed"].apply(lambda x: [y for y in x if not y in cachedStopWords and y and y[0] != "#"])


In [467]:
# Find most common words
top_words = [i[0] for i in Counter(tweets["clean_words"].explode()).most_common(10)]

# remove any words that are not in the top dataset

# Group data by week
tweets["top_words"] = tweets["clean_words"].apply(lambda x: list(set(x if x else []) & set(top_words)))
word_timeline = tweets.groupby([pd.Grouper(key="timestamp", freq="W-MON")])["top_words"].sum().apply(lambda x: x if isinstance(x, list) else []).apply(Counter)

# reduce dataset to timeline with word mentions
out = pd.DataFrame([(t, k, v) for t, d in zip(word_timeline.index.values, word_timeline) for k, v in d.items()])
out.columns = ['Date', 'Word', 'Count']

fig = px.line(out, x="Date", y="Count", color="Word")
fig.write_image(os.path.join(full_export_dir, "top_words.png"))
fig.show()

## Locations
Distribution of tweets per location

In [35]:
import pycountry

def get_iso_3166(country: str):
    try:
        res = pycountry.countries.search_fuzzy(country)[0].alpha_3
    except LookupError:
        return
    return res

[(c, v) for c, v in Counter(tweets_with_user["country"]).most_common() if c].sort()

In [52]:
import pycountry

top_countries = pd.DataFrame([(c, v) for c, v in Counter(tweets_with_user["country"]).most_common(30) if c])
top_countries.columns = ['country', 'tweets']

top_countries["ISO_alpha"] = top_countries["country"].apply(lambda x: get_iso_3166(x))
top_countries["legend"] = top_countries[["country", "tweets"]].apply(lambda x: " - ".join(x.astype(str)), axis=1)


fig_gs = px.scatter_geo(top_countries, locations="ISO_alpha", size="tweets")

# fig_gs.update_traces(showlegend=True, mode="markers+text")
fig_gs.update_geos(fitbounds="locations",  resolution=110, visible=True, showframe=False)
fig_gs.update_layout(
    title_text = 'Tweet Distribution by user location',
    autosize=False,
    margin=dict(l=10,
                r=10,
                b=0,
                t=50,
                pad=4,
                autoexpand=False
            )
)
fig.write_image(os.path.join(full_export_dir, "top_locations_bubble.png"))
fig_gs.show()

In [47]:
fig_bar = px.bar(top_countries, x="tweets", y="country", color="country", labels={"tweets": "Number of Tweets", "country": "Country"})

fig_bar.update_layout(
    title_text = 'Tweet Distribution by user location',
    autosize=False,
)

fig_map = px.choropleth(top_countries, locations="ISO_alpha",
                    color="tweets",
                    hover_name="country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.amp,
                    labels={"tweets": "Tweets"},
                    )

fig_map.update_geos(fitbounds="locations",  resolution=110, visible=True, showframe=False)
fig_map.update_layout(
    title_text = 'Tweet Distribution by user location',
    autosize=False,
    margin=dict(l=0,
                r=100,
                b=0,
                t=50,
                pad=4,
                autoexpand=False
            )
)

fig.write_image(os.path.join(full_export_dir, "top_locations_bar.png"))
fig.write_image(os.path.join(full_export_dir, "top_locations_map.png"))
fig_map.show()
fig_bar.show()

## Retweets Distribution
Distribution of retweets per day for the top subset 

In [474]:
top_retweeted = tweets.sort_values(by=["retweets"], ascending=False)[0:int(len(tweets)*0.05)].sort_index()
retweets = top_retweeted.groupby([pd.Grouper(key="timestamp", freq="D")])["retweets"].apply(list).reset_index().explode("retweets").fillna(0)
fig = px.box(retweets, 
             x="timestamp",
             y="retweets",
             title='Distribution of retweets per day for the top subset',
             labels={
                "timestamp": "Date",
                "retweets": "Retweets"
                },
                log_y=True
                )

fig.write_image(os.path.join(full_export_dir, "retweet_distribution.png"))
fig.show()


## Replies Distribution
Distribution of replies per day for the top subset

In [473]:

top_replied = tweets.sort_values(by=["replies"], ascending=False)[0:int(len(tweets)*0.05)].sort_index()

replies = top_replied.groupby([pd.Grouper(key="timestamp", freq="D")]).replies.apply(list).reset_index().explode("replies").fillna(0)


fig = px.box(replies, x="timestamp", y="replies",             
             title='Distribution of replies per day for the top subset',
             labels={
                "timestamp": "Date",
                "replies": "Replies"
                },
                log_y=True)

fig.write_image(os.path.join(full_export_dir, "replies_distribution.png"))
fig.show()

## Total Values

In [472]:
import plotly.graph_objects as go

words = tweets["tweet_text"].apply(lambda x: re.sub(re_pattern, "", x)).str.split(" |_|-").explode()

data = [
    ("Unique tweets",tweets.__len__()),
    ("Geolocated tweets", len(tweets[~tweets.location.isnull()].location)),
    ("Tweets with media", len(tweets[tweets[["images", "videos"]].sum(axis=1) > 1])),
    ("Tweets with URLs", len(tweets[tweets["urls"].str.len() > 1 ])),
    ("Words", len(words)),
    ("Unique Words", len(Counter(words))),
    ("Hashtags ", len(tweets["hashtags"].explode())),
    ("Unique Hashtags ", len(Counter(tweets["hashtags"].explode()))),
]

table = go.Table(header=dict(values=["", 'Values']),
                 cells=dict(values=[[k for k, v in data], [v for k,v in data]], align="left")
                 )

fig = go.Figure(data=[table])
fig.write_image(os.path.join(full_export_dir, "Totals.png"))
fig.show()